In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\khanh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import ast
import os

In [3]:
# Đọc dữ liệu từ file CSV
file_path = 'Data/HondaVietnam.csv'
df = pd.read_csv(file_path)

In [4]:
# Hàm để chuyển giá trị từ reaction_count sang reactions dưới dạng từ điển
def convert_to_dict(row):
    if pd.isna(row['reactions']):
        try:
            # Nếu giá trị là chuỗi, xử lý chuỗi định dạng từ điển
            if isinstance(row['reaction_count'], str):
                reaction_count = ast.literal_eval(row['reaction_count'])
            else:
                # Nếu giá trị là số nguyên, sử dụng nó trực tiếp
                reaction_count = {'like': row['reaction_count'], 'love': 0, 'wow': 0}

            return {'like': reaction_count.get('like', 0),
                    'love': reaction_count.get('love', 0),
                    'wow': reaction_count.get('wow', 0)}
        except (SyntaxError, ValueError):
            # Xử lý nếu có lỗi khi đánh giá biểu thức Python
            return {'like': 0, 'love': 0, 'wow': 0}
    else:
        return ast.literal_eval(row['reactions'])

# Áp dụng hàm cho mỗi dòng trong dataframe
df['reactions'] = df.apply(convert_to_dict, axis=1)


In [5]:
# Thay thế giá trị bị thiếu ở cột 'text'
df['text'] = df['text'].fillna('HondaVietnam' + df['post_id'].astype(str))
# Thay thế giá trị bị thiếu ở cột 'post_text' bằng giá trị của cột 'text'
df['post_text'] = df['post_text'].fillna(df['text'])
# thay thế các giá trị bị thiếu ỏ cột shared_text
df['shared_text'] = df['shared_text'].fillna(df['text'])
#thay thế 
df['reactors'] = df['reactors'].fillna('[]')
df['w3_fb_url'] = df['w3_fb_url'].fillna('https://www.facebook.com/HondaVietnam/posts/910761847075317')

In [6]:
# Xóa cột thiếu data
df = df.drop(columns=['original_text'])
df = df.drop(columns=['video_duration_seconds'])
df = df.drop(columns= ['video_height'])
df = df.drop(columns= ['video_size_MB'])
df = df.drop(columns= ['video_watches'])
df = df.drop(columns= ['video_width'])

In [7]:
# Đường dẫn đầy đủ của file đầu ra
output_file_path = os.path.join('Data', 'HondaVietnam_cleaned.csv')

# Lưu lại file sau khi làm sạch
df.to_csv(output_file_path, index=False)